In [46]:
from IPython.display import display, Math, Latex
from IPython.core.display import HTML
from IPython.display import display, HTML
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import apyori
from apyori import apriori
import shutil, os
import json
import pyfpgrowth

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

### Apriori Algorithm

Рассматривается дерево префиксов (prefix tree), где 2 элемента $X$ и $Y$ соединены, если $X$ является прямым подмножеством $Y$. 

Apriori алгоритм использует следующее утверждение: если $X \subseteq Y$, то $supp(X) \geq supp(Y)$. Откуда следуют следующие 2 свойства:
 -  если $Y$ встречается часто, то любое подмножество $X: X \subseteq Y$  так же встречается часто
 -  если $X$ встречается редко, то любое супермножество $Y: Y \supseteq X$ так же встречается редко
 
Apriori алгоритм поуровнево проходит по префиксному дереву и рассчитывает частоту встречаемости подмножеств $X$ в $D$. Таким образом, в соответствии с алгоритмом:
 -  исключаются редкие подмножества и все их супермножества
 -  рассчитывается $supp(X)$ для кадого подходящего кандидата $X$ размера $k$ на уровне $k$
 
Таким образом, Apriori-алгоритм сначала ищет все единичные (содержащие 1 элемент) itemsets, удовлетворяющие заданному пользователем 𝑠𝑢𝑝𝑝, затем составляет из них пары по принципу иерархической монотонности, т.е. если 𝑥1 встречается часто и 𝑥2 встречается часто, то и [𝑥1,𝑥2] встречается часто.

In [25]:
dataset = pd.read_csv('Market_Basket_Optimisation.csv', header = None)

In [26]:
dataset.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,whole wheat pasta,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,soup,light cream,shallot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,frozen vegetables,spaghetti,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
#заполняем нан значения реальными
dataset.fillna(method = 'ffill',axis = 1, inplace = True)

In [28]:
dataset.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs
2,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney
3,turkey,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado
4,mineral water,milk,energy bar,whole wheat rice,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea
5,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt,low fat yogurt
6,whole wheat pasta,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries
7,soup,light cream,shallot,shallot,shallot,shallot,shallot,shallot,shallot,shallot,shallot,shallot,shallot,shallot,shallot,shallot,shallot,shallot,shallot,shallot
8,frozen vegetables,spaghetti,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea
9,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries,french fries


In [29]:
transactions = [] #матрица транзакций
for i in range(0, 7501): 
    transactions.append([str(dataset.values[i,j]) for j in range(0, 20)])

Support - показатель "частотности" данного itemset во всех анализируемых транзакциях.

Confidence - показатель того, как часто наше правило срабатывает для всего датасета. (Если случилось Х, то случится и У)

Lift - показатель того, насколько items зависят друг от друга (Если lift=1, то итемы независимы и правило не работает, Если больше 1, то итемы зависимы (чем больше число - тем сильнее)

In [50]:
%%time
# значения задаем самостоятельно:
# min_support -- минимальный support для правил 
# min_confidence -- минимальное значение confidence для правил 
# min_lift -- минимальный lift 
# max_length -- максимальная длина itemset 
result = list(apyori.apriori(transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 4, min_length = 2))

Wall time: 673 ms


In [35]:
output = []
for RelationRecord in result:
    o = StringIO()
    apyori.dump_as_json(RelationRecord, o)
    output.append(json.loads(o.getvalue()))
data_df = pd.DataFrame(output)

In [39]:
pd.set_option('display.max_colwidth', None)
display(HTML(data_df.to_html()))

,items,support,ordered_statistics
0,"[chicken, light cream]",0.004533,"[{'items_base': ['light cream'], 'items_add': ['chicken'], 'confidence': 0.29059829059829057, 'lift': 4.84395061728395}]"
1,"[escalope, pasta]",0.005866,"[{'items_base': ['pasta'], 'items_add': ['escalope'], 'confidence': 0.3728813559322034, 'lift': 4.700811850163794}]"
2,"[fromage blanc, honey]",0.003333,"[{'items_base': ['fromage blanc'], 'items_add': ['honey'], 'confidence': 0.2450980392156863, 'lift': 5.164270764485569}]"
3,"[olive oil, whole wheat pasta]",0.007999,"[{'items_base': ['whole wheat pasta'], 'items_add': ['olive oil'], 'confidence': 0.2714932126696833, 'lift': 4.122410097642296}]"
4,"[pasta, shrimp]",0.005066,"[{'items_base': ['pasta'], 'items_add': ['shrimp'], 'confidence': 0.3220338983050847, 'lift': 4.506672147735896}]"
5,"[cake, frozen vegetables, tomatoes]",0.003066,"[{'items_base': ['cake', 'frozen vegetables'], 'items_add': ['tomatoes'], 'confidence': 0.2987012987012987, 'lift': 4.367560314928736}]"
6,"[cereals, ground beef, spaghetti]",0.003066,"[{'items_base': ['cereals', 'spaghetti'], 'items_add': ['ground beef'], 'confidence': 0.45999999999999996, 'lift': 4.681763907734057}]"
7,"[chocolate, ground beef, herb & pepper]",0.003999,"[{'items_base': ['chocolate', 'herb & pepper'], 'items_add': ['ground beef'], 'confidence': 0.4411764705882354, 'lift': 4.4901827759597746}]"
8,"[eggs, ground beef, herb & pepper]",0.004133,"[{'items_base': ['eggs', 'ground beef'], 'items_add': ['herb & pepper'], 'confidence': 0.2066666666666667, 'lift': 4.178454627133872}]"
9,"[french fries, ground beef, herb & pepper]",0.003200,"[{'items_base': ['french fries', 'ground beef'], 'items_add': ['herb & pepper'], 'confidence': 0.23076923076923078, 'lift': 4.665768194070081}, {'items_base': ['french fries', 'herb & pepper'], 'items_add': ['ground beef'], 'confidence': 0.46153846153846156, 'lift': 4.697421981004071}]"


- Пары items
- items_base - первый элемент пары
- items_add - второй (добавленный алгоритмом) элемент пары
- confidence - значение confidence для пары
- lift - значение lift для пары
- support - начение support для пары.

### Eclat

Идея алгоритма ECLAT (Equivalence CLAss Transformation) заключается в ускорении подсчета 𝑠𝑢𝑝𝑝(𝑋), т.к. для его подсчета в Apriori надо просканировать весь датасет.

Если $t(X)$ обозначает множество всех транзакций, где встречается подмножество $X$, то
<br>$t(XY) = t(X) \cap t(Y)$</br>
<br>и</br>
<br>$supp(XY) = |t(XY)|$</br>
<br> то есть $supp(XY)$ равен размеру множества $t(XY)$

ECLAT производит поиск в глубину в отличие от "горизонтального" для Apriori.

Вначале генерируется пустое множество I, это позволяет нам на первом проходе выделить все частотные itemsets. Затем алгоритм будет вызывать сам себя и увеличивать I на 1 на каждом шаге до тех пор, пока не будет достигнута заданная пользователем длина I.
Для хранения значений используется префиксное дерево. Вначале строится нулевой корень дерева (пустое множество I), затем по мере прохода по itemsets алгоритм прописывает содержащиеся в каждом itesmsets items, при этом самая левая ветвь является child нулевого корня и далее вниз. При этом ветвей столько, сколько items встречаестя в itemsets. Такой подход позволяет записывать itemset в памяти только один раз, что делает ECALT быстрее Apriori.


In [51]:
class Eclat:
    #инициализация объекта класса
    def __init__(self, min_support = 0.01, max_items = 5, min_items = 2):
        self.min_support = min_support
        self.max_items = max_items
        self.min_items = min_items
        self.item_lst = list()
        self.item_len = 0
        self.item_dict = dict()
        self.final_dict = dict()
        self.data_size = 0
    
    #создание словаря из ненулевых объектов из всех транзакций (вертикальный датасет)
    def read_data(self, dataset):
        for index, row in dataset.iterrows():
            row_wo_na = row.dropna().unique()
            for item in row_wo_na:
                item = item.strip()
                if item in self.item_dict:
                    self.item_dict[item][0] += 1
                else:
                    self.item_dict.setdefault(item, []).append(1)
                self.item_dict[item].append(index)
        #задаем переменные экземпляра (instance variables)
        self.data_size = dataset.shape[0]
        self.item_lst = list(self.item_dict.keys())
        self.item_len = len(self.item_lst)
        self.min_support = self.min_support * self.data_size
        #print ("min_supp", self.min_support)
        
    #рекурсивный метод для поиска всех ItemSet по алгоритму Eclat
    #структура данных: {Item: [Supp number, tid1, tid2, tid3, ...]}
    def recur_eclat(self, item_name, tids_array, minsupp, num_items, k_start):
        if tids_array[0] >= minsupp and num_items <= self.max_items:
            for k in range(k_start+1, self.item_len):
                if self.item_dict[self.item_lst[k]][0] >= minsupp:
                    new_item = item_name + " | " + self.item_lst[k]
                    new_tids = np.intersect1d(tids_array[1:], self.item_dict[self.item_lst[k]][1:])
                    new_tids_size = new_tids.size
                    new_tids = np.insert(new_tids, 0, new_tids_size)
                    if new_tids_size >= minsupp:
                        if num_items >= self.min_items: self.final_dict.update({new_item: new_tids})
                        self.recur_eclat(new_item, new_tids, minsupp, num_items+1, k)
    
    #последовательный вызов функций определенных выше
    def fit(self, dataset):
        i = 0
        self.read_data(dataset)
        for w in self.item_lst:
            self.recur_eclat(w, self.item_dict[w], self.min_support, 2, i)
            i+=1
        return self
        
    #вывод в форме словаря {ItemSet: support(ItemSet)}
    def transform(self):
        return {k: "{0:.4f}%".format((v[0]+0.0)/self.data_size*100) for k, v in self.final_dict.items()}

In [52]:
model = Eclat(min_support = 0.01, max_items = 4, min_items = 3)

In [53]:
model.fit(dataset)

In [54]:
model.transform()

{'mineral water | olive oil | spaghetti': '1.0265%',
 'mineral water | eggs | milk': '1.3065%',
 'mineral water | eggs | spaghetti': '1.4265%',
 'mineral water | eggs | chocolate': '1.3465%',
 'mineral water | eggs | ground beef': '1.0132%',
 'mineral water | milk | frozen vegetables': '1.1065%',
 'mineral water | milk | spaghetti': '1.5731%',
 'mineral water | milk | chocolate': '1.3998%',
 'mineral water | milk | ground beef': '1.1065%',
 'mineral water | french fries | spaghetti': '1.0132%',
 'mineral water | frozen vegetables | spaghetti': '1.1998%',
 'mineral water | spaghetti | chocolate': '1.5865%',
 'mineral water | spaghetti | pancakes': '1.1465%',
 'mineral water | spaghetti | ground beef': '1.7064%',
 'mineral water | chocolate | ground beef': '1.0932%',
 'eggs | spaghetti | chocolate': '1.0532%',
 'milk | spaghetti | chocolate': '1.0932%'}